<a href="https://colab.research.google.com/github/yohanesnuwara/reservoir-engineering/blob/master/Unit%2010%20Gas-Condensate%20Reservoirs/notebook/10_problems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Unit 10. Gas-Condensate Reservoirs**

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
!git clone https://github.com/yohanesnuwara/reservoir-engineering

Cloning into 'reservoir-engineering'...
remote: Enumerating objects: 226, done.
remote: Counting objects: 100% (226/226), done.
remote: Compressing objects: 100% (219/219), done.
remote: Total 1135 (delta 108), reused 0 (delta 0), pack-reused 909
Receiving objects: 100% (1135/1135), 16.87 MiB | 24.20 MiB/s, done.
Resolving deltas: 100% (503/503), done.


## Problem 10.1

In [15]:
def cvd_condensate(z, z2, temp, p, Gp, Np, Vo):
    """
    Calculate volatile oil-gas ratio of condensate from Constant-Volume Depletion (CVD) Study
    Walsh and Towler (1995)

    Inputs
    z: measured gas-phase compressibility factor (array)
    z2: measured two-phase compressibility factor (array)
    p: measured pressure (array)
    Gp: gas produced in the PVT cell
    Np: condensate produced in the PVT cell
    Vo: condensate volume in the PVT cell
    """

    z_j = z; Gp_j = Gp; Np_j = Np; z2_j = z2; Vo_j = Vo
    j = p

    # calculate gas FVF (Bg)
    Bg = (0.00503676 * z_j * (temp + 460)) / p  # in RB/scf

    # initial gas FVF
    Bgi = Bg[0]

    # initial Gfg
    Gfgi = Gp_j[-1]  # in scf

    # initial Nfo
    Nfoi = Np_j[-1]

    # calculate initial Vtg (Vtg1)
    Vtg1 = Gfgi * Bgi  # in res bbl

    # initial values for Eq 10.14
    ntj_nt1 = 1
    delta_ngj_to_nt1 = 0

    # empty arrays for appending
    Vtoj_arr = []
    ntj_nt1_arr = []
    Vtoj_Vtgj_arr = []
    Vtgj_arr = []
    delta_Vtgj_arr = []
    ngj_nt1_arr = []
    delta_ngj_to_ngj_arr = []
    delta_ngj_to_nt1_arr = []
    delta_Gpj_arr = []
    delta_Npj_arr = []
    Gfgj_arr = []
    Nfgj_arr = []
    Gj_arr = []
    Nj_arr = []
    Gfoj_arr = []
    Nfoj_arr = []
    Boj_arr = []
    Bgj_arr = []
    Rsj_arr = []
    Rvj_arr = []

    for i in range(len(j) - 1):

        # Eq 10.13
        Vtoj = Vo_j[i] * Vtg1
        Vtoj_arr.append(Vtoj)

        # Eq 10.14
        ntj_nt1 = ntj_nt1 - delta_ngj_to_nt1
        ntj_nt1_arr.append(ntj_nt1)

        # Eq 10.15
        Vtoj_Vtgj = ((Vtg1 * z2_j[i] * p[0]) / (z2_j[0] * p[i])) * (ntj_nt1)
        Vtoj_Vtgj_arr.append(Vtoj_Vtgj)

        # Eq 10.16
        Vtgj = Vtoj_Vtgj - Vtoj
        Vtgj_arr.append(Vtgj)

        # Eq 10.17
        delta_Vtgj = Vtoj_Vtgj - Vtg1
        delta_Vtgj_arr.append(delta_Vtgj)

        # Eq 10.18
        ngj_nt1 = (Vtgj * z_j[0] * p[i]) / (z_j[i] * Vtg1 * p[0])
        ngj_nt1_arr.append(ngj_nt1)

        # Eq 10.19
        delta_ngj_to_ngj = delta_Vtgj / Vtgj
        delta_ngj_to_ngj_arr.append(delta_ngj_to_ngj)

        # Eq 10.20
        delta_ngj_to_nt1 = delta_ngj_to_ngj * ngj_nt1
        delta_ngj_to_nt1_arr.append(delta_ngj_to_nt1)

        if i == 0:
            # Eq 10.21
            delta_Gpj = Gp_j[i] - 0

            # Eq 10.22
            delta_Npj = Np_j[i] - 0

            # Eq 10.23
            Gj = Gfgi - delta_Gpj

            # Eq 10.24
            Nj = Nfoi - delta_Gpj

            # Eq 10.25
            Gfgj = Gfgi

            # Eq 10.26
            Nfgj = Nfoi

        if i > 0:
            # Eq 10.21
            delta_Gpj = Gp_j[i] - Gp_j[i - 1]

            # Eq 10.22
            delta_Npj = Np_j[i] - Np_j[i - 1]

            # Eq 10.23
            Gj = Gj - delta_Gpj_arr[-1]

            # Eq 10.24
            Nj = Nj - delta_Npj_arr[-1]

            # Eq 10.25
            Gfgj = (Vtgj * delta_Gpj) / delta_Vtgj

            # Eq 10.26
            Nfgj = (Vtgj * delta_Npj) / delta_Vtgj

        delta_Gpj_arr.append(delta_Gpj)
        delta_Npj_arr.append(delta_Npj)
        Gj_arr.append(Gj)
        Nj_arr.append(Nj)
        Gfgj_arr.append(Gfgj)
        Nfgj_arr.append(Nfgj)

        # Eq 10.27
        Gfoj = Gj - Gfgj
        Gfoj_arr.append(Gfoj)

        # Eq 10.28
        Nfoj = Nj - Nfgj
        Nfoj_arr.append(Nfoj)

        # Eq 10.29
        Boj = Vtoj / Nfoj
        Boj_arr.append(Boj)

        # Eq 10.30
        Bgj = Vtgj / Gfgj
        Bgj_arr.append(Bgj)

        # Eq 10.31
        Rsj = Gfoj / Nfoj
        Rsj_arr.append(Rsj)

        # Eq 10.32
        Rvj = (Nfgj / Gfgj) * 1E+06  # result in STB/scf
        Rvj_arr.append(Rvj)

    Rv = Rvj_arr
    return(Rv)

In [6]:
columns = ['j', 'p', 'T', 'Gp', 'Np', 'z', 'z2', 'Vo']
data = pd.read_csv('/content/reservoir-engineering/Unit 10 Gas-Condensate Reservoirs/data/Table 10.13-CVD Results for Problem 10.1.csv', names=columns)
data

,j,p,T,Gp,Np,z,z2,Vo
0,1,5450.0,215,0.00,0.00,1.0439,1.0439,0.0000
1,2,5300.0,215,11.51,1.82,1.0217,1.0269,0.0500
2,3,4800.0,215,61.64,7.53,0.9552,0.9779,0.1850
3,4,4300.0,215,119.02,12.64,0.9033,0.9311,0.2176
4,5,3800.0,215,190.70,17.31,0.8648,0.8927,0.2381
5,6,3300.0,215,272.80,21.28,0.8384,0.8590,0.2431
6,7,2800.0,215,366.85,24.57,0.8264,0.8320,0.2407
7,8,2300.0,215,469.94,27.15,0.8300,0.8093,0.2346
8,9,1800.0,215,579.24,29.22,0.8466,0.7868,0.2245
9,10,1300.0,215,690.53,30.89,0.8744,0.7531,0.2123


In [23]:
z = data['z'].values
z2 = data['z2'].values
p = data['p'].values
Gp = data['Gp'].values * 1E+3 # convert Mscf to scf
Np = data['Np'].values
Vo = data['Vo'].values
temp = data['T'].iloc[0]

Rv = cvd_condensate(z, z2, temp, p, Gp, Np, Vo)

result = pd.DataFrame({'p': p[:-1], 'T': data['T'].iloc[:-1], 'Gp': Gp[:-1], 
                       'Np': Np[:-1], 'z': z[:-1], 'z2': z2[:-1], 'Vo': Vo[:-1], 'Rv': Rv})
result

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:147: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:155: RuntimeWarning: invalid value encountered in double_scalars


,p,T,Gp,Np,z,z2,Vo,Rv
0,5450.0,215,0.0,0.00,1.0439,1.0439,0.0000,165.500000
1,5300.0,215,11510.0,1.82,1.0217,1.0269,0.0500,158.123371
2,4800.0,215,61640.0,7.53,0.9552,0.9779,0.1850,113.903850
3,4300.0,215,119020.0,12.64,0.9033,0.9311,0.2176,89.055420
4,3800.0,215,190700.0,17.31,0.8648,0.8927,0.2381,65.150670
5,3300.0,215,272800.0,21.28,0.8384,0.8590,0.2431,48.355664
6,2800.0,215,366850.0,24.57,0.8264,0.8320,0.2407,34.981393
7,2300.0,215,469940.0,27.15,0.8300,0.8093,0.2346,25.026676
8,1800.0,215,579240.0,29.22,0.8466,0.7868,0.2245,18.938701
9,1300.0,215,690530.0,30.89,0.8744,0.7531,0.2123,15.005841
